# Setup
## <font color=brown>Environment and imports</font>

### <font color=blue>Paths & Imports</font>

In [ ]:
#Paths
import os, math, json, time, sys, getpass
if os.name=='posix':
    rootDir = '/Users/edrazor/OneDrive/CurrentWork/AgnostiQ/IBM-DQC1/'
elif os.name=='nt':
    rootDir = '/Users/Ed Gonzalez/OneDrive/CurrentWork/AgnostiQ/IBM-DQC1/'

os.chdir(rootDir + 'src/')

#IPython imports option
%load_ext autoreload
%autoreload 2  #Refresh modules upon change

#IBM QASM imports
from IBMQuantumExperience import IBMQuantumExperience
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, QISKitError, qasm, IBMQ, compile

#Numpy / Scipy / Matplotlib etc.
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
%matplotlib inline
import itertools
from scipy import linalg as la
from pprint import pprint

#Current project imports
import utils
import DQC13twirlrandomized as b3

In [ ]:
from qiskit import providers

### <font color=blue>IBM token registration</font>

In [ ]:
#Setup IBM token
APItoken = "Input key here."
qx_url = "https://quantumexperience.ng.bluemix.net/api"

ibmqe = IBMQ.enable_account(APItoken, qx_url)

# 3-Braid section
## <font color=brown>Circuit definitions and compilation</font>

In [ ]:
layoutIndices = [
    [ 1, 0],
    [13, 1],
    [ 1, 2],
    [13,12],
    [12, 2],
    [ 2, 3],
    [11,12],
    [11, 3],
    [ 4, 3],
    [11,10],
    [ 4,10],
    [ 5, 4],
    [ 9,10],
    [ 5, 9],
    [ 5, 6],
    [ 9, 8],
    [ 6, 8],
    [ 7, 8]
]

def get_layout(indices):
    return {("qr",0):("qr",indices[0]),("qr",1):("qr",indices[1])}

In [ ]:
shots = 1024
reps = 10  #Number of times to evaluate a polynomial.
pollingT = 60  #How frequently to poll IBM (in secs) while waiting for job to complete.

# =============Knot selection===========#
writhe= np.array(1*[0,1,2,3,4,5,6,7,8,9])
#opSeq = ['s23']
opSeq = ['s12']

norm  = b3.getNorm(writhe)

# ===========Backend selection==========#
#backend = IBMQ.get_backend('ibmq_qasm_simulator')
#backend = IBMQ.get_backend('ibmqx5')
backend = IBMQ.get_backend('ibmq_16_melbourne')
#backend = IBMQ.get_backend('ibmqx4')
#backend = IBMQ.get_backend('ibmqx2')

# ======Compile for target backend======#
#compiledJob = b3.buildJob(opSeq, shots, backend, reps)
compiledJob = b3.buildJob(opSeq, shots, backend, reps, get_layout(layoutIndices[17]))
compiledJobArray = []
for idx in range(5):
    compiledJobArray.append(compiledJob)

### <font color=blue>Execute Job!</font>

In [ ]:
resultsArray = []
for idx in range(5):
    runjobs = backend.run(compiledJobArray[idx])
    print("Init stat, #",idx, ": ", runjobs.status())

    flagDone = False
    idx = 0
    while not flagDone:
        time.sleep(pollingT)
        flagDone = (runjobs.status() == providers.JobStatus.DONE)
        print("Status: ", runjobs.status(), ", Queue position is ", runjobs.queue_position())
        idx += 1
    resultsArray.append(runjobs.result())

## <font color=brown>Collect results</font>

In [ ]:
def padDict_2Q(resDict):
    """
    Pad dictionary of resulting counts from a 2-qubit experiment
    By default, if there are no counts in a particular computational basis,
    that result is not added as a key in the dictionary, making subsequent
    addition difficult
    """
    bases = ['00','01','10','11']
    for basis in bases:
        count = resDict.get(basis)
        if count == None:
            resDict.update({basis:0})

def addDict_2Q(res1,res2):
    """
    Take two dictionary of counts from IBMQ.
    Adds counts in the second to the first.
    """
    padDict_2Q(res1)
    padDict_2Q(res2)
    
    for basis in res2:
        res1[basis] += res2[basis]

In [ ]:
def computeJones(rawResults, rep):
    """
    Takes a 'results' object from a QPU or QASM simulator.
    Returns Jones' polynomial evaluation for the "rep"-th evaluation.
    """
    #Retrieve weight 1 counts
    arrState = np.array(['o','x'])
    arrBasis = np.array(['x','y'])
    arrZipped = utils.zipTuple(arrBasis,arrState)
    countW1 = 4*['']
    idx = 0
    for i in arrZipped:
        name = "Rep" + str(rep) + ",Weight 1," + i[0] + i[1]
        countW1[idx] = rawResults.get_counts(name)
        idx += 1

    #Retrieve weight 2 counts
    arrState = np.array(['x'])
    arrBasis = np.array(['x','y'])
    arrZipped= utils.zipTuple(arrBasis,arrState)
    countW2 = 2*['']
    idx = 0
    for i in arrZipped:
        name = "Rep" + str(rep) + ",Weight 2," + i[0] + i[1]
        countW2[idx] = rawResults.get_counts(name)
        idx += 1

    return -b3.computeJones(countW1, countW2)*norm[rep]

In [ ]:
def computeJonesConcat(rawResultsArray, rep):
    """
    Assuming we ran the experiment for 18 different layout maps on the QPU,
    concatenate all 18 experiments for a given twirl number.
    """
    #Retrieve weight 1 counts
    arrState = np.array(['o','x'])
    arrBasis = np.array(['x','y'])
    arrZipped = utils.zipTuple(arrBasis,arrState)
    countW1 = 4*['']
    idx = 0
    for i in arrZipped:
        for layout in range(18):
            name = "Rep" + str(rep) + ",Weight 1," + i[0] + i[1]
            if layout == 0:
                countTemp = rawResultsArray[layout].get_counts(name).copy()
            else:
                addDict_2Q(countTemp,rawResultsArray[layout].get_counts(name))
        countW1[idx] = countTemp.copy()
        idx += 1

    #Retrieve weight 2 counts
    arrState = np.array(['x'])
    arrBasis = np.array(['x','y'])
    arrZipped= utils.zipTuple(arrBasis,arrState)
    countW2 = 2*['']
    idx = 0
    for i in arrZipped:
        for layout in range(18):
            name = "Rep" + str(rep) + ",Weight 2," + i[0] + i[1]
            if layout == 0:
                countTemp = rawResultsArray[layout].get_counts(name).copy()
            else:
                addDict_2Q(countTemp,rawResultsArray[layout].get_counts(name))
        countW2[idx] = countTemp.copy()
        idx += 1

    return -b3.computeJones(countW1, countW2)*norm[rep]

### <font color=blue>Compute and print out Jones' polynomial evaluations</font>

In [ ]:
jonesVals = []
for rep in range(reps):
    jonesVals.append(computeJonesConcat(resultsArray, rep))
    print(jonesVals[rep])

In [ ]:
for idx in range(5):
    results = resultsArray[idx]
    jonesVals = reps*['']
    for rep in range(reps):
        jonesVals[rep] = computeJones(results, rep)
        print(jonesVals[rep])

# Repeat once

In [ ]:
shots = 1024
reps = 10  #Number of times to evaluate a polynomial.
pollingT = 60  #How frequently to poll IBM (in secs) while waiting for job to complete.

# =============Knot selection===========#
writhe= np.array(3*[0,1,2,3,4,5,6,7,8,9])
#opSeq = ['s23']
opSeq = ['s12']

norm  = b3.getNorm(writhe)

# ===========Backend selection==========#
#backend = IBMQ.get_backend('ibmq_qasm_simulator')
#backend = IBMQ.get_backend('ibmqx5')
backend = IBMQ.get_backend('ibmq_16_melbourne')
#backend = IBMQ.get_backend('ibmqx4')
#backend = IBMQ.get_backend('ibmqx2')

# ======Compile for target backend======#
#compiledJob = b3.buildJob(opSeq, shots, backend, reps)
compiledJobArray = []
for idx in range(18):
    compiledJobArray.append(b3.buildJob(opSeq, shots, backend, reps, get_layout(layoutIndices[idx])))

In [ ]:
resultsArray = []
for idx in range(18):
    runjobs = backend.run(compiledJobArray[idx])
    print("Init stat, #",idx, ": ", runjobs.status())

    flagDone = False
    idx = 0
    while not flagDone:
        time.sleep(pollingT)
        flagDone = (runjobs.status() == providers.JobStatus.DONE)
        print("Status: ", runjobs.status(), ", Queue position is ", runjobs.queue_position())
        idx += 1
    resultsArray.append(runjobs.result())

In [ ]:
jonesVals = []
for rep in range(reps):
    jonesVals.append(computeJonesConcat(resultsArray, rep))
    print(jonesVals[rep])

In [ ]:
for idx in range(18):
    results = resultsArray[idx]
    jonesVals = reps*['']
    for rep in range(reps):
        jonesVals[rep] = computeJones(results, rep)
        print(jonesVals[rep])

### <font color=blue>Trash code to print out raw counts for troubleshooting</font>

In [ ]:
countW1tr = 2*['']
idx = 0
for i in countW2:
    countW1tr[idx] = utils.traceGetCount(i, [1])
    idx += 1

print(countW1tr)

In [ ]:
countW1tr = 4*['']
idx = 0
for i in countW1:
    countW1tr[idx] = utils.traceGetCount(i, [1])
    idx += 1

print(countW1tr)

### <font color=blue>Print backend low-level stats</font>

In [ ]:
backend = IBMQ.get_backend('ibmq_16_melbourne')
nqubits = 14
params = backend.properties()['qubits']

pulse_length = nqubits*[[]]
buffer_length = nqubits*[[]]
t1times = nqubits*[[]]
t2times = nqubits*[[]]

for idx in range(nqubits):
    pulse_length[idx] = [params[idx]['gateTime']['value'],params[idx]['gateTime']['unit']] # single-qubit gate time
    buffer_length[idx] = [params[idx]['buffer']['value'],params[idx]['buffer']['unit']] # spacing between pulses
    t1times[idx] = [params[idx]['T1']['value'],params[idx]['T1']['unit']] # spacing between pulses
    t2times[idx] = [params[idx]['T2']['value'],params[idx]['T2']['unit']] # spacing between pulses

# Changelog
From v0.1, modified syntactically to handle deprecation of a bunch of stuff in qiskit 0.6.
- Mainly, QuantumProgram() is deprecated. Instead, compile() takes a LIST of QuantumCircuit().
    - QuantumCircuit(), in addition to taking QuantumRegister and ClassicalRegister, also takes 'names='
    - These same names are used for count retrieval.
- backend sent to compile() is no-longer a string but an object retrieved with IBMQ.get_backend("backend name")